In [ ]:
!pip install bitsandbytes
!pip install trl
!pip install peft
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 38.8 MB/s eta 0:00:00


In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [ ]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [ ]:
model_id = "google/gemma-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, token=os.environ['HF_TOKEN'])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
text = "What are legal lawsuits,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=250)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

What are legal lawsuits, and what are the different types of legal lawsuits?

Legal lawsuits are legal actions that are taken against a person or company for a wrong or injury that has been caused.

The different types of legal lawsuits are:

1. Breach of contract
2. Breach of warranty
3. Negligence
4. Fraud
5. Product liability
6. Personal injury
7. Wrongful death
8. Medical malpractice
9. Employment discrimination
10. Consumer protection

<h2><strong>Breach of contract</strong></h2>

A breach of contract is a legal action that is taken against a person or company for failing to fulfill their obligations under a contract.

<h2><strong>Breach of warranty</strong></h2>

A breach of warranty is a legal action that is taken against a person or company for failing to fulfill their obligations under a warranty.

<h2><strong>Negligence</strong></h2>

Negligence is a legal action that is taken against a person or company for failing to exercise reasonable care in their actions.

<h2><strong>F

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM"
)

In [ ]:
from datasets import load_dataset

In [ ]:
stream = load_dataset("AnuragB/Indian-legal", split="train", streaming=True)
small = list(stream.take(15000))

Resolving data files:   0%|          | 0/7130 [00:00<?, ?it/s]

In [ ]:
print(len(small))

15000


In [ ]:
small[0]

{'text': 'Appeal No. LXVI of 1949.'}

In [ ]:
small[14999]

{'text': 'Sec tions 83A and 83 B of the Indian Companies Act can only be supported as valid on the ground that they regulate the management of companies and are, therefore, within the said entry.'}

In [ ]:
def tokenize_fn(examples):
    outputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,
        return_overflowing_tokens=False
    )
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs

In [ ]:
tokenized_dataset = [tokenize_fn(x) for x in small]
print(len(tokenized_dataset))

15000


In [ ]:
from transformers import TrainingArguments, Trainer
from peft import get_peft_model

In [ ]:
training_args = TrainingArguments(
    output_dir="./gemma-cpt-legal",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=150,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=50,
    bf16=True
)


In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 25,001,984 || all params: 8,562,682,880 || trainable%: 0.2920


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: madhukarjai9 (madhukarjai9-thapar-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.112000
20,2.886700
30,2.657000
40,2.668400
50,2.613600
60,2.740300
70,2.621900
80,2.451400
90,2.686300
100,2.514200


TrainOutput(global_step=150, training_loss=2.598929748535156, metrics={'train_runtime': 1615.1684, 'train_samples_per_second': 0.371, 'train_steps_per_second': 0.093, 'total_flos': 1124959109806080.0, 'train_loss': 2.598929748535156, 'epoch': 0.04})

In [ ]:
text = "Appeal from the High Court of judicature, Bombay, in a reference under section,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=250)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Appeal from the High Court of judicature, Bombay, in a reference under section, 529 of the Indian Penal Code. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal dismissed. Appeal d

In [ ]:
import torch
from datasets import load_dataset
import math

eval_data = small[:100]

def compute_perplexity(model, tokenizer, dataset):
    model.eval()
    total_loss = 0
    count = 0

    for item in dataset:
        enc = tokenizer(item["text"], return_tensors="pt", truncation=True, max_length=512)
        enc = {k: v.to(model.device) for k,v in enc.items()}
        with torch.no_grad():
            loss = model(**enc, labels=enc["input_ids"]).loss
        total_loss += loss.item()
        count += 1

    avg_loss = total_loss / count
    ppl = math.exp(avg_loss)
    return avg_loss, ppl

loss, ppl = compute_perplexity(model, tokenizer, eval_data)
print("Evaluation Loss:", loss)
print("Perplexity:", ppl)


Evaluation Loss: 2.6548352360725405
Perplexity: 14.22264248783265


In [ ]:
from peft import PeftModel

model_id = "google/gemma-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, token=os.environ['HF_TOKEN'])

lora_path = "./gemma-cpt-legal/checkpoint-150"

merged_model = PeftModel.from_pretrained(
    base_model,
    lora_path
)

merged_model = merged_model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
os.environ["HF_TOKEN_WRITE"] = userdata.get("HF_TOKEN_WRITE")
os.environ["HF_TOKEN_REPO"] = userdata.get("HF_TOKEN_REPO")

In [ ]:
merged_model.save_pretrained("merged-model")
tokenizer.save_pretrained("merged-model")

('merged-model/tokenizer_config.json',
 'merged-model/special_tokens_map.json',
 'merged-model/tokenizer.model',
 'merged-model/added_tokens.json',
 'merged-model/tokenizer.json')

In [ ]:
from huggingface_hub import whoami, HfApi
print(whoami())

{'type': 'user', 'id': '674b62961e1ca74a4121fac0', 'name': 'jaimadhukar', 'fullname': 'Jai Madhukar', 'email': 'madhukarjai9@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/28b9b9c36b69f49a11c575c8d84ca4be.svg', 'orgs': [{'type': 'org', 'id': '69176f0e5c9c18a331f20174', 'name': 'jaimadhukar007', 'fullname': 'Jai Madhukar', 'email': 'madhukarjai9@gmail.com', 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://www.gravatar.com/avatar/a4ce9981e44d4b4db84007037e5680ac?d=retro&size=100', 'roleInOrg': 'admin', 'isEnterprise': False}], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'HF_TOKEN', 'role': 'read', 'createdAt': '2025-11-14T16:00:15.370Z'}}}


In [ ]:

merged_model.push_to_hub("jaimadhukar/lawvista", token=os.environ["HF_TOKEN_REPO"])
tokenizer.push_to_hub("jaimadhukar/lawvista", token=os.environ["HF_TOKEN_REPO"])

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0002-of-00002.safetensors:   0%|          |  610kB /  934MB            

  ...0001-of-00002.safetensors:   0%|          |  623kB / 5.00GB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...pg_n4ys0h/tokenizer.model:   0%|          | 16.2kB / 4.24MB            

  ...mpg_n4ys0h/tokenizer.json:  73%|#######3  | 25.1MB / 34.4MB            

CommitInfo(commit_url='https://huggingface.co/jaimadhukar/lawvista/commit/f7faea133ece393101f95c614ff6f70c407f8e7b', commit_message='Upload tokenizer', commit_description='', oid='f7faea133ece393101f95c614ff6f70c407f8e7b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jaimadhukar/lawvista', endpoint='https://huggingface.co', repo_type='model', repo_id='jaimadhukar/lawvista'), pr_revision=None, pr_num=None)